# ws 06

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = sns.load_dataset('iris')
# df.head()
df.sample(5, random_state=1)

In [ ]:
df.isnull().sum()

In [ ]:
sns.scatterplot(x='petal_length', y='petal_width', data=df, hue='species', style='species', 
                alpha=0.85, edgecolor='w', s=100)
plt.show()

In [ ]:
y, class_names = pd.factorize(df.species, sort=True) 
# print(pd.unique(y))

class_names, y[45:105]

In [ ]:
class_names, y = np.unique(df.species, return_inverse=True)
class_names, y[45:105]

In [ ]:
X = df.drop('species', axis=1)
# X.head(6)
X.sample(6, random_state=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_sc = sc.fit_transform(X)
X_sc[:4]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=0.25, random_state=1) #

X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape 

In [ ]:
y_train[:5]

In [ ]:
from tensorflow.keras.utils import to_categorical

def one_hot_encode(arr):
    categories, ids = np.unique(arr, return_inverse=True)
    return to_categorical(ids, len(categories))

y_train_1h = one_hot_encode(y_train).astype('int')
y_test_1h = one_hot_encode(y_test).astype('int')

y_test_1h[:4]

In [ ]:
y_train_1h = pd.get_dummies(y_train)
y_test_1h = pd.get_dummies(y_test)
y_test_1h.head()

In [ ]:
y_test_1h[:5] 

In [ ]:
y_train_1h.shape, y_test_1h.shape 

In [ ]:
X_train[:4]

In [ ]:
y_train[:4]

In [ ]:
y_train_1h[:4]

# Model

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense 


model = Sequential()

hidden_node = 64
 
model.add(Dense(hidden_node, activation='relu', input_shape=(4,))) 

model.add(Dense(3, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.output_shape

In [ ]:
model.get_config()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train_1h, epochs=100, batch_size=8, verbose=1, validation_split=0.25) 

In [ ]:
score = model.evaluate(X_test, y_test_1h, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
model.metrics_names

In [ ]:
y_pred = model.predict(X_test)

y_pred[:4]
y_pred[:4].round(3)

In [ ]:
y_pred_cat = np.argmax(y_pred, axis=1)
y_pred_cat[:4]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred_cat, target_names=class_names))
print(confusion_matrix(y_test, y_pred_cat))

In [ ]:
df_hist = pd.DataFrame.from_dict(history.history)

df_hist['loss'].plot(style='b--', label='Trainning')
df_hist['val_loss'].plot(style='r-', label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.legend()
plt.title('Loss')
plt.show()

In [ ]:
df_hist['accuracy'].plot(style='b--', label='Trainning')
df_hist['val_accuracy'].plot(label='Test')

plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
X_new = [[8.1, 4.0, 5.9, 2.1], # 
         [6.8, 3.6, 3.5, 1.4]] # 

# X_new = [[8.1, 4.0, 5.9, 2.1]]

X_new_sc = sc.transform(X_new)
X_new_sc

In [ ]:
y_pred_pr = model.predict(X_new_sc)
y_pred = np.argmax(y_pred_pr, axis=1)
print(class_names[y_pred])

## Boundary

In [ ]:
class Onehot2Int(object):
    def __init__(self, model):
        self.model = model

    def predict(self, X):
        y_pred = self.model.predict(X)
        return np.argmax(y_pred, axis=1)
    
model_no_ohe = Onehot2Int(model)    

In [ ]:
from mlxtend.plotting import plot_decision_regions

value = 0
width = 5.5

ax = plot_decision_regions(X_test, np.array(y_test),

                    clf=model_no_ohe, feature_index=[2, 3],  
                    filler_feature_values={0: value, 1: value},
                    filler_feature_ranges={0: width, 1: width},
                    legend=2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, class_names, framealpha=0.5)

plt.title('Iris (Hidden={})'.format(hidden_node))
# plt.xticks([])
plt.yticks([])
plt.xlabel('petal_length')
plt.ylabel('petal_width')

plt.scatter(X_new_sc[:,2][y_pred==0], X_new_sc[:,3][y_pred==0], marker='s', s=180, c='b')
plt.scatter(X_new_sc[:,2][y_pred==1], X_new_sc[:,3][y_pred==1], marker='^', s=230, c='#7d4128') # 
plt.scatter(X_new_sc[:,2][y_pred==2], X_new_sc[:,3][y_pred==2], marker='o', s=150, c='b')
plt.show()